In [ ]:
import mne
import sys

sys.path.append('..')

from _parameters import *
sys.path.remove('..')


In [ ]:
def open_tfr(s, moment, modal):
    
    # Dict per cond
    tfr = dict.fromkeys(cond[modal], None)

    # Read tfr per cond
    for c in cond[modal]:

        tfr_fname = dirs['tfr'] + '/tfr_' + moment + '_' + c.replace('/', '_') + '_s' + str(s) + '.h5'    

        # Add to dict
        tfr[c] = mne.time_frequency.read_tfrs(fname=tfr_fname)[0]

    return tfr

In [ ]:
tfr_vis = open_tfr(s=1, moment = 'enc1', modal = 'visual')